In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error



In [2]:
# Read the CSV file
df = pd.read_csv('dataformodeltraining1.csv')

In [3]:
df.columns

Index(['LocationID', 'date', 'time', 'day_type', 'holiday', 'busyness', 'temp',
       'weather_description'],
      dtype='object')

In [4]:
df.head

<bound method NDFrame.head of         LocationID   date      time day_type  holiday  busyness  temp  \
0                4  01-17  00:00:00  workday        1      5.00   2.8   
1               13  01-17  00:00:00  workday        1      1.50   2.8   
2               24  01-17  00:00:00  workday        1      0.75   2.8   
3               41  01-17  00:00:00  workday        1      3.50   2.8   
4               42  01-17  00:00:00  workday        1      2.25   2.8   
...            ...    ...       ...      ...      ...       ...   ...   
510553         239  12-01  16:00:00  workday        0      0.50   5.6   
510554         249  12-01  16:00:00  workday        0      0.50   5.6   
510555         236  12-01  09:00:00  workday        0      1.00   3.3   
510556         237  12-01  09:00:00  workday        0      0.50   3.3   
510557         249  12-01  01:00:00  workday        0      0.50   4.4   

       weather_description  
0                     Snow  
1                     Snow  
2     

In [5]:
df['time'] = pd.to_datetime(df['time']).dt.hour * 3600 + pd.to_datetime(df['time']).dt.minute * 60 + pd.to_datetime(df['time']).dt.second
df['weather_description'] = pd.Categorical(df['weather_description'])
df['weather_description'] = df['weather_description'].cat.codes

/var/folders/6q/9wmlbvpj5nqfn_clctmzyck00000gn/T/ipykernel_1799/3101677476.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time']).dt.hour * 3600 + pd.to_datetime(df['time']).dt.minute * 60 + pd.to_datetime(df['time']).dt.second
/var/folders/6q/9wmlbvpj5nqfn_clctmzyck00000gn/T/ipykernel_1799/3101677476.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time']).dt.hour * 3600 + pd.to_datetime(df['time']).dt.minute * 60 + pd.to_datetime(df['time']).dt.second
/var/folders/6q/9wmlbvpj5nqfn_clctmzyck00000gn/T/ipykernel_1799/3101677476.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`.

In [6]:
# Encode categorical columns using one-hot encoding
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
cat_columns = ['date', 'day_type']
encoded_cols = encoder.fit_transform(df[cat_columns])
encoded_df = pd.DataFrame(encoded_cols)

# Create unique feature names for the encoded columns
feature_names = []
for i, column in enumerate(cat_columns):
    unique_categories = encoder.categories_[i]
    for category in unique_categories:
        feature_names.append(f'{column}_{category}')

encoded_df.columns = feature_names

df = pd.concat([df.drop(columns=cat_columns), encoded_df], axis=1)


/Users/saoirse/miniconda3/envs/mayaisawesome/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
print(df.dtypes)

LocationID            int64
time                  int32
holiday               int64
busyness            float64
temp                float64
                     ...   
date_11-29          float64
date_11-30          float64
date_12-01          float64
day_type_weekend    float64
day_type_workday    float64
Length: 343, dtype: object


In [16]:
df.shape

(510558, 343)

In [8]:
# Select the relevant columns for X (features)
X = df.drop(columns=['busyness'])

# Select the target variable y (busyness)
y = df['busyness']

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Convert the data into DMatrix format, which is required by XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [11]:
# # Set the parameters for the XGBoost model
# params = {
#     'max_depth': 3,
#     'eta': 0.1,
#     'objective': 'binary:logistic',
#     'eval_metric': 'error'
# }

In [12]:

# Set the parameters for XGBoost
params = {
    'objective': 'reg:squarederror',
    'eta': 0.1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Train the XGBoost model
num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

In [14]:

# Make predictions on the test set
y_pred = model.predict(dtest)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

RMSE: 16.383051684617325
Mean Absolute Error: 10.371957791547338
Root Mean Squared Error: 16.383051684617325
